2. Простой пример на Python: логгер‑одиночка

In [1]:
from __future__ import annotations
from datetime import datetime
from threading import Lock
from typing import Optional

class Logger:
    """
    Потокобезопасный логгер‑одиночка.
    Первый вызов Logger.instance() создаёт объект,
    последующие — возвращают его же.
    """
    _instance: Optional["Logger"] = None
    _lock = Lock()                     # защита от гонок

    def __init__(self, file: str = "app.log") -> None:
        self._file = open(file, "a", encoding="utf‑8")

    @classmethod
    def instance(cls) -> "Logger":
        if cls._instance is None:      # 1‑я проверка (быстрая)
            with cls._lock:
                if cls._instance is None:      # 2‑я (безопасная)
                    cls._instance = cls()
        return cls._instance

    def log(self, msg: str) -> None:
        timestamp = datetime.now().isoformat(timespec="seconds")
        self._file.write(f"[{timestamp}] {msg}\n")
        self._file.flush()

# ---------- Клиентский код ----------
if __name__ == "__main__":
    logger1 = Logger.instance()
    logger2 = Logger.instance()
    logger1.log("Привет из первого места")
    logger2.log("И из второго — тот же объект")

    print(logger1 is logger2)          # True


True


3. Сложный пример: конфигурация приложения + горячее обновление
Сценарий: большое веб‑приложение читает YAML‑конфиг при старте.
Хотим (1) иметь один экземпляр конфига, (2) уметь перезагружать его без рестарта, (3) не ломать работающие потоки.

In [4]:
from __future__ import annotations
import yaml, time, threading
from pathlib import Path
from typing import Any, Dict, Optional

class Config:
    _instance: Optional["Config"] = None
    _lock = threading.RLock()

    def __init__(self,
                 path: str = "settings.yaml",
                 defaults: Optional[Dict[str, Any]] = None) -> None:
        self._path = Path(path)
        self._data: Dict[str, Any] = defaults or {}
        self._mtime = 0.0
        self._ensure_file()        # <— создаём файл при необходимости
        self._load()

        self._watcher = threading.Thread(
            target=self._watch_file, daemon=True, name="ConfigWatcher"
        )
        self._watcher.start()

    # ---------- Singleton access ----------
    @classmethod
    def instance(cls, **kwargs) -> "Config":
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = cls(**kwargs)
        return cls._instance

    # ---------- Public API ----------
    def get(self, key: str, default: Any = None) -> Any:
        with self._lock:
            return self._data.get(key, default)

    def reload(self) -> None:
        with self._lock:
            self._load()

    # ---------- Internal helpers ----------
    def _ensure_file(self) -> None:
        """Если файла нет — создаём его с текущими defaults."""
        if not self._path.exists():
            self._path.write_text(
                yaml.safe_dump(self._data, sort_keys=False) or "# empty config\n",
                encoding="utf‑8",
            )

    def _load(self) -> None:
        with self._path.open(encoding="utf‑8") as f:
            loaded = yaml.safe_load(f) or {}
        self._data.update(loaded)           # merge, чтобы не потерять defaults
        self._mtime = self._path.stat().st_mtime
        print("⚙️  Конфиг перезагружен")

    def _watch_file(self) -> None:
        while True:
            time.sleep(1.0)
            try:
                mtime = self._path.stat().st_mtime
            except FileNotFoundError:
                continue
            if mtime != self._mtime:
                self.reload()

# ---------- Демонстрация ----------
if __name__ == "__main__":
    cfg = Config.instance(defaults={"app_name": "MyService"})
    print("App:", cfg.get("app_name"))
    print("DB:", cfg.get("database", {}))
    time.sleep(10)


⚙️  Конфиг перезагружен
App: MyService
DB: {'host': 'localhost', 'port': 5432, 'user': 'app', 'password': 'secret'}
